In [1]:
import pandas as pd
import numpy as np
import re
import string
import urllib.request
import requests
import bs4 as bs
from nltk.corpus import stopwords
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import  train_test_split
from keras.models import Sequential,Model
from keras.layers import LSTM, Dense, Bidirectional, Input,Dropout,BatchNormalization,CuDNNLSTM, GRU, CuDNNGRU, Embedding, GlobalMaxPooling1D, GlobalAveragePooling1D, Flatten
from keras.layers import SpatialDropout1D
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# building a crawler to get insomnia data 

In [ ]:
def getlinks(soup):
    l = []
    for link in soup.find_all('a',class_ = "bbp-topic-permalink",href = True):
        l.append(link['href'][0:])
        
    return l

def getdate(soup):
    l = []
    div = soup.find('span', class_ = 'bbp-reply-post-date')
    return(div.text)
#
def getname(soup):
    l = []
    div = soup.find('a', class_ = 'bbp-author-name') 
    return([div.text])   
#
def gettext(soup):
    p = soup.find_all('div', class_ = 'bbp-reply-content')
    return p[1].text
#

sitelist = []
site = 'https://insomniacoach.com/forums/id/insomnia-help'
string = '/page/'
hdr = {'User-Agent': 'Mozilla/5.0'}
for i in range(1,197): 
    sitelist.append(site + string + str(i)+'/')

dftoappend = pd.DataFrame()
df = pd.DataFrame()

for sites in sitelist:
    print(sites)
    request = Request(sites, headers=hdr)
    sauce = urllib.request.urlopen(request)
    soup = bs.BeautifulSoup(sauce,'html.parser')
    links = getlinks(soup)
    for link in links:
        request1 = Request(link, headers=hdr)
        sauce = urllib.request.urlopen(request1)
        soup = bs.BeautifulSoup(sauce,'html.parser')
        date = getdate(soup)
        name = getname(soup)
        text = gettext(soup)
        d = [{'DATE' : date, 'NAME' : name, 'TEXT' : text, 'STATE' : "insomnia"}]
        dftoappend = pd.DataFrame(d)
        df = df.append(dftoappend,ignore_index = True)
df.to_csv('insomnia.csv')

# loading the other data 

In [2]:
anxiety=pd.read_csv('anxiety.csv')
depression=pd.read_csv('depression.csv')
trauma=pd.read_csv('trauma.csv')
suicidal_thoughts=pd.read_csv('suicidal_thoughts.csv')
stress=pd.read_csv('stress.csv')
neutral=pd.read_csv('neutral.csv')
insomnia=pd.read_csv('insomnia.csv')

In [5]:
df=pd.concat([anxiety,depression,trauma,suicidal_thoughts,stress,insomnia,neutral])
df

,Unnamed: 0.1,Unnamed: 0,DATE,NAME,TEXT,STATE
0,0.0,0.0,\n \n ...,['CMF'],\nThere is a thread under Depression just like...,anxiety
1,1.0,1.0,\n \n ...,['annabay'],"\nhey there, what physical symptoms of anxiety...",anxiety
2,2.0,2.0,\n \n ...,['AGrace'],"\nHi Everyone,\nHere are some ideas for helpin...",anxiety
3,3.0,3.0,\n \n ...,['Lind779'],"\nHi everyone,\nI would love to hear your stor...",anxiety
4,4.0,4.0,\n \n ...,['Chris B'],"\nHi everyone, particularly any new members wh...",anxiety
...,...,...,...,...,...,...
33650,NaN,94448.0,none,['y11611'],"They have an alright casino area, food wasn't ...",neutral
33651,NaN,94449.0,none,"['y1234', 'y1235']","We went around lunchtime, but it wasn't very b...",neutral
33652,NaN,94450.0,none,"['y2418', 'y8084']",We ordered the MTO breakfast with a pancake.,neutral
33653,NaN,94452.0,none,"['y18958', 'y4005']",The only people in Vegas that aren't predators...,neutral


# cleaning the data

In [6]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str(text).lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['TEXT'] = df['TEXT'].apply(clean_text)
df['TEXT'] = df['TEXT'].str.replace('\d+', '')

C:\Users\waels\AppData\Local\Temp/ipykernel_9812/3427233334.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df['TEXT'] = df['TEXT'].str.replace('\d+', '')


In [7]:
df.drop(columns = ['Unnamed: 0'],inplace = True)
df.drop(columns = ['Unnamed: 0.1'],inplace = True)

In [8]:
df

,DATE,NAME,TEXT,STATE
0,\n \n ...,['CMF'],thread depression like thought start aniety on...,anxiety
1,\n \n ...,['annabay'],hey physical symptoms aniety eperience often f...,anxiety
2,\n \n ...,['AGrace'],hi everyone ideas helping manage symptoms anie...,anxiety
3,\n \n ...,['Lind779'],hi everyone would love hear stories overcome a...,anxiety
4,\n \n ...,['Chris B'],hi everyone particularly new members may readi...,anxiety
...,...,...,...,...
51406,none,['y11611'],alright casino area food wasnt great rooms clean,neutral
51407,none,"['y1234', 'y1235']",went around lunchtime wasnt busy,neutral
51408,none,"['y2418', 'y8084']",ordered mto breakfast pancake,neutral
51409,none,"['y18958', 'y4005']",people vegas arent predators moving game,neutral


# tokenizing the data

In [9]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['TEXT'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 68552 unique tokens.


In [10]:
X = tokenizer.texts_to_sequences(df['TEXT'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (51411, 250)


In [11]:
Y = pd.get_dummies(df['STATE']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (51411, 7)


In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(46269, 250) (46269, 7)
(5142, 250) (5142, 7)


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();


In [ ]:
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show();